In [1]:
import fastrepl.repl as fastrepl

fastrepl.LLMCache.disable()
fastrepl.DEBUG(0)

In [2]:
from IPython.display import clear_output

In [3]:
from datasets import load_dataset

# We have dataset for meta-eval
dataset = load_dataset("yelp_review_full", split="test")
dataset = dataset.shuffle(seed=8)
dataset = dataset.select(range(30))
dataset = dataset.rename_column("text", "input")
dataset = dataset.map(
    lambda row: {"reference": row["label"] + 1, "input": row["input"]},
    remove_columns=["label"],
)

In [4]:
def print_metric(metric_name, predictions, references):
    metric = fastrepl.load_metric(metric_name)
    result = metric.compute(predictions=predictions, references=references)
    print(f"{metric_name}: {result[metric_name]}")

In [5]:
clear_output(wait=True)

eval1 = fastrepl.Evaluator(
    pipeline=[
        fastrepl.LLMGradingHead(
            model="gpt-3.5-turbo",
            context="You will get a input text from Yelp review. Grade user's satisfaction from 1 to 5.",
            number_from=1,
            number_to=5,
            position_debias_strategy="shuffle",
        )
    ]
)


def to_int(example):
    example["prediction"] = int(example["prediction"])
    return example


result1 = fastrepl.LocalRunner(evaluator=eval1, dataset=dataset).run()
result1 = result1.map(to_int)

print_metric("accuracy", result1["prediction"], result1["reference"])
print_metric("mse", result1["prediction"], result1["reference"])
print_metric("mae", result1["prediction"], result1["reference"])

result1.to_pandas()

Output()

/Users/yujonglee/dev/fastrepl/fastrepl/fastrepl/warnings.py:24: CompletionTruncatedWarning: 5 | 
https://docs.fastrepl.com/miscellaneous/warnings_and_errors#completiontruncated

/Users/yujonglee/dev/fastrepl/fastrepl/fastrepl/warnings.py:24: CompletionTruncatedWarning: 4 | 
https://docs.fastrepl.com/miscellaneous/warnings_and_errors#completiontruncated

/Users/yujonglee/dev/fastrepl/fastrepl/fastrepl/warnings.py:24: CompletionTruncatedWarning: 1 | 
https://docs.fastrepl.com/miscellaneous/warnings_and_errors#completiontruncated

/Users/yujonglee/dev/fastrepl/fastrepl/fastrepl/warnings.py:24: CompletionTruncatedWarning: 3 | 
https://docs.fastrepl.com/miscellaneous/warnings_and_errors#completiontruncated

/Users/yujonglee/dev/fastrepl/fastrepl/fastrepl/warnings.py:24: CompletionTruncatedWarning: 2 | 
https://docs.fastrepl.com/miscellaneous/warnings_and_errors#completiontruncated

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

accuracy: 0.4
mse: 1.8
mae: 0.9333333333333333


,input,reference,prediction
0,Stayed at the Wm Penn down the street for a we...,5,5
1,We tried out the lunch specials and found them...,4,5
2,Should have known better than to eat in a plac...,1,4
3,This place has sure changed...and not for the ...,2,1
4,I've been bringing my son to the owner Michael...,5,5
5,The trip to the location takes two busses and ...,1,1
6,This restaurant was suggested to me by a frien...,4,4
7,This place is unique because you are sitting o...,3,5
8,"Whew, $10 for a whiskey ginger ale and $18 for...",3,1
9,"Loove me some Pei Wei! Boyfriend isn't a fan, ...",4,1


In [7]:
clear_output(wait=True)

# simple references + COT
eval2 = fastrepl.Evaluator(
    pipeline=[
        fastrepl.LLMGradingHead(
            model="gpt-3.5-turbo",
            context="You will get a input text from Yelp review. Grade user's satisfaction from 1 to 5.",
            number_from=1,
            number_to=5,
            position_debias_strategy="shuffle",
            references=[("this is the best!", "5"), ("this is the worst!", "1")],
        )
    ]
)


def to_int(example):
    example["prediction"] = int(example["prediction"])
    return example


result2 = fastrepl.LocalRunner(evaluator=eval2, dataset=dataset).run()
result2 = result2.map(to_int)

print_metric("accuracy", result2["prediction"], result2["reference"])
print_metric("mse", result2["prediction"], result2["reference"])
print_metric("mae", result2["prediction"], result2["reference"])

result2.to_pandas()

Output()

/Users/yujonglee/dev/fastrepl/fastrepl/fastrepl/warnings.py:24: CompletionTruncatedWarning: 2 | 
https://docs.fastrepl.com/miscellaneous/warnings_and_errors#completiontruncated

/Users/yujonglee/dev/fastrepl/fastrepl/fastrepl/warnings.py:24: CompletionTruncatedWarning: 4 | 
https://docs.fastrepl.com/miscellaneous/warnings_and_errors#completiontruncated

/Users/yujonglee/dev/fastrepl/fastrepl/fastrepl/warnings.py:24: CompletionTruncatedWarning: 5 | 
https://docs.fastrepl.com/miscellaneous/warnings_and_errors#completiontruncated

/Users/yujonglee/dev/fastrepl/fastrepl/fastrepl/warnings.py:24: CompletionTruncatedWarning: 1 | 
https://docs.fastrepl.com/miscellaneous/warnings_and_errors#completiontruncated

/Users/yujonglee/dev/fastrepl/fastrepl/fastrepl/warnings.py:24: CompletionTruncatedWarning: 3 | 
https://docs.fastrepl.com/miscellaneous/warnings_and_errors#completiontruncated

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

accuracy: 0.7
mse: 0.3
mae: 0.3


,input,reference,prediction
0,Stayed at the Wm Penn down the street for a we...,5,4
1,We tried out the lunch specials and found them...,4,4
2,Should have known better than to eat in a plac...,1,2
3,This place has sure changed...and not for the ...,2,2
4,I've been bringing my son to the owner Michael...,5,5
5,The trip to the location takes two busses and ...,1,1
6,This restaurant was suggested to me by a frien...,4,4
7,This place is unique because you are sitting o...,3,3
8,"Whew, $10 for a whiskey ginger ale and $18 for...",3,3
9,"Loove me some Pei Wei! Boyfriend isn't a fan, ...",4,4


In [7]:
clear_output(wait=True)

eval3 = fastrepl.Evaluator(
    pipeline=[
        fastrepl.LLMGradingHeadCOT(
            model="gpt-3.5-turbo",
            context="You will get a input text from Yelp review. Grade user's satisfaction from 1 to 5.",
            number_from=1,
            number_to=5,
        )
    ]
)


def to_int(example):
    if example["prediction"] is None:
        example["prediction"] = 0
    example["prediction"] = int(example["prediction"])
    return example


result3 = fastrepl.LocalRunner(evaluator=eval3, dataset=dataset).run()
result3 = result3.map(to_int)

print_metric("accuracy", result3["prediction"], result3["reference"])
print_metric("mse", result3["prediction"], result3["reference"])
print_metric("mae", result3["prediction"], result3["reference"])

result3.to_pandas()

Output()

/Users/yujonglee/dev/fastrepl/fastrepl/fastrepl/warnings.py:24: CompletionTruncatedWarning: ### Thoughts
The reviewer starts by mentioning that the restaurant was suggested to them by a friend. They comment on the 
seating arrangement and describe the restaurant as "sexy and dark", although they mention that it was hard to read 
the menu in such lighting. They praise the vibe of the restaurant and highlight the extraordinary wine list. The 
reviewer then talks about the food they ordered, specifically mentioning the sausage flatbread, pesto cream pasta, 
and the hamachi, all of which they found to be incredible, flavorful, and fresh. They mention that the flatbread 
was surprisingly spicy and perfect, given that the restaurant is not specifically known for pizza. However, they 
did not have dessert so they cannot comment on that. They then mention that the service started off great but 
became less attentive as the night went on, requiring them to flag down another waiter for the check. In 
conclusion, the reviewer states that they plan on coming back to the restaurant, as it is a great place to go with 
a group or | https://docs.fastrepl.com/miscellaneous/warnings_and_errors#completiontruncated

/Users/yujonglee/dev/fastrepl/fastrepl/fastrepl/warnings.py:24: InvalidPredictionWarning: 'r' not in ['1', '2', 
'3', '4', '5']. full: ### Thoughts
The reviewer starts by mentioning that the restaurant was suggested to them by a friend. They comment on the 
seating arrangement and describe the restaurant as "sexy and dark", although they mention that it was hard to read 
the menu in such lighting. They praise the vibe of the restaurant and highlight the extraordinary wine list. The 
reviewer then talks about the food they ordered, specifically mentioning the sausage flatbread, pesto cream pasta, 
and the hamachi, all of which they found to be incredible, flavorful, and fresh. They mention that the flatbread 
was surprisingly spicy and perfect, given that the restaurant is not specifically known for pizza. However, they 
did not have dessert so they cannot comment on that. They then mention that the service started off great but 
became less attentive as the night went on, requiring them to flag down another waiter for the check. In 
conclusion, the reviewer states that they plan on coming back to the restaurant, as it is a great place to go with 
a group or | https://docs.fastrepl.com/miscellaneous/warnings_and_errors#invalidprediction

/Users/yujonglee/dev/fastrepl/fastrepl/fastrepl/warnings.py:24: InvalidPredictionWarning: '8' not in ['1', '2', 
'3', '4', '5']. full: ### Thoughts
The reviewer mentions that the restaurant has a nice classic American decor, specifically calling out the American 
farmer and Route 66 style. This suggests that the ambiance of the place is appealing and nostalgic. The reviewer 
also mentions that the food was very good and the service was really good too, indicating a positive dining 
experience. Additionally, the fact that the restaurant is family-owned and that they care about their customers 
speaks to their commitment and dedication. Finally, the reviewer states that they will definitely be coming back, 
which further implies their satisfaction with the overall experience.
### Result
4.8 | https://docs.fastrepl.com/miscellaneous/warnings_and_errors#invalidprediction

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

accuracy: 0.7
mse: 1.5666666666666667
mae: 0.5666666666666667


,input,reference,prediction
0,Stayed at the Wm Penn down the street for a we...,5,4
1,We tried out the lunch specials and found them...,4,4
2,Should have known better than to eat in a plac...,1,2
3,This place has sure changed...and not for the ...,2,2
4,I've been bringing my son to the owner Michael...,5,5
5,The trip to the location takes two busses and ...,1,1
6,This restaurant was suggested to me by a frien...,4,0
7,This place is unique because you are sitting o...,3,3
8,"Whew, $10 for a whiskey ginger ale and $18 for...",3,3
9,"Loove me some Pei Wei! Boyfriend isn't a fan, ...",4,5
